In [41]:

# Importation des librairies
import pymongo
import pandas as pd

# Connexion à la base de données
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Récupérer le nom de la base de données
db = client["moviesimdb"]

# Récupérer le nom de la collection
collection = db["movies_detail"]

# Récupérer les documents
documents = collection.find()

# Afficher les documents
for document in documents:
    print(document)

{'_id': ObjectId('639c31409265665d123efcb7'), 'title': 'The Intouchables', 'original_title': 'Original title: Intouchables', 'score': '8.5', 'genre': ['Biography', 'Comedy', 'Drama', 'Back to top'], 'date': '2011', 'synopsis': 'After he becomes a quadriplegic from a paragliding accident, an aristocrat hires a young man from the projects to be his caregiver.', 'duree': 112, 'casting': ['François Cluzet', 'Omar Sy', 'Anne Le Ny']}
{'_id': ObjectId('639c31409265665d123efcb8'), 'title': 'Grave of the Fireflies', 'original_title': 'Original title: Hotaru no haka', 'score': '8.5', 'genre': ['Animation', 'Drama', 'War', 'Back to top'], 'date': '1988', 'synopsis': 'A young boy and his little sister struggle to survive in Japan during World War II.', 'duree': 89, 'casting': ['Tsutomu Tatsumi', 'Ayano Shiraishi', 'Akemi Yamaguchi']}
{'_id': ObjectId('639c31409265665d123efcb9'), 'title': 'Rear Window', 'original_title': None, 'score': '8.5', 'genre': ['Mystery', 'Thriller', 'Back to top'], 'date'

In [42]:
film_plus_long = db.movies_detail.find().sort([('duree', pymongo.DESCENDING)]).limit(1)[0]
film_plus_long


{'_id': ObjectId('639c314d9265665d123efd21'),
 'title': 'Gone with the Wind',
 'original_title': None,
 'score': '8.2',
 'genre': ['Drama', 'Romance', 'War', 'Back to top'],
 'date': '1939',
 'synopsis': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.',
 'duree': 238,
 'casting': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell']}

In [43]:
films_mieux_notes = db.movies_detail.find().sort([('score', pymongo.DESCENDING)]).limit(5)
films_mieux_notes

In [44]:
title_film = []
score_film = []

for film in films_mieux_notes:
    title_film.append(film['title'])
    score_film.append(film['score'])

df_meilleurs_films = pd.DataFrame({'title': title_film, 'score': score_film})
df_meilleurs_films


,title,score
0,The Shawshank Redemption,9.3
1,The Godfather,9.2
2,The Lord of the Rings: The Return of the King,9.0
3,The Dark Knight,9.0
4,12 Angry Men,9.0


In [45]:
morgan_freeman_movies= collection.aggregate([

    {
        "$match":({"casting":"Morgan Freeman"})
    },
    {
        "$count" : "Morgan Freeman"
    }
    
])

for i in morgan_freeman_movies:
    print(i)

{'Morgan Freeman': 4}


In [46]:
tom_cruise_movies= collection.aggregate([

    {
        "$match":({"casting":"Tom Cruise"})
    },
    {
        "$count" : "Tom Cruise"
    }
    
])

for i in tom_cruise_movies:
    print(i)

{'Tom Cruise': 1}


In [47]:
meilleurs_films_dhorreur = db.movies_detail.find({"genre":"Horror"}).sort([('score', pymongo.DESCENDING)]).limit(3)


In [48]:
title_film = []
score_film = []
 
for film in meilleurs_films_dhorreur:
    title_film.append(film['title'])
    score_film.append(film['score'])

df_meilleurs_films_dhorreur = pd.DataFrame({'title': title_film, 'score': score_film})
df_meilleurs_films_dhorreur

,title,score
0,Psycho,8.5
1,Alien,8.5
2,The Shining,8.4


In [49]:
meilleurs_films_drama = db.movies_detail.find({"genre":"Drama"}).sort([('score', pymongo.DESCENDING)]).limit(3)


In [50]:
title_film = []
score_film = []
 
for film in meilleurs_films_drama:
    title_film.append(film['title'])
    score_film.append(film['score'])

df_meilleurs_films_drama = pd.DataFrame({'title': title_film, 'score': score_film})
df_meilleurs_films_drama

,title,score
0,The Shawshank Redemption,9.3
1,The Godfather,9.2
2,Schindler's List,9.0


In [51]:
meilleurs_films_comique = db.movies_detail.find({"genre":"Comedy"}).sort([('score', pymongo.DESCENDING)]).limit(3)

In [52]:
title_film = []
score_film = []
 
for film in meilleurs_films_comique:
    title_film.append(film['title'])
    score_film.append(film['score'])

df_meilleurs_films_comique = pd.DataFrame({'title': title_film, 'score': score_film})
df_meilleurs_films_comique

,title,score
0,Life Is Beautiful,8.6
1,City Lights,8.5
2,The Intouchables,8.5


In [54]:
pourcentage_film_americain= collection.aggregate([
    {
        "$sort": { "score":-1}       
    },
    {
        "$limit": 100
    },
    {
        "$match":({"pays":"United States"})
    },
    {
        "$count" : "United States"
    }
    
])

for i in pourcentage_film_americain:
    print(i)


{'United States': 75}


In [63]:
pipe = [
    {
        "$unwind":"$genre"
    },
    {
        '$group': {
            '_id': "$genre",
            'mean': {"$avg": "$duree"}
        }
    }
]

In [64]:
list_a = []
for movie in db.movies_detail.aggregate(pipe):
    list_a.append(movie)

list_a

[{'_id': 'Biography', 'mean': 140.9655172413793},
 {'_id': 'History', 'mean': 156.0},
 {'_id': 'Family', 'mean': 114.53846153846153},
 {'_id': 'Sport', 'mean': 98.4},
 {'_id': 'Musical', 'mean': 103.0},
 {'_id': 'Crime', 'mean': 128.31372549019608},
 {'_id': 'War', 'mean': 134.6818181818182},
 {'_id': 'Fantasy', 'mean': 123.57142857142857},
 {'_id': 'Back to top', 'mean': 126.27710843373494},
 {'_id': 'Film-Noir', 'mean': 110.0},
 {'_id': 'Horror', 'mean': 120.6},
 {'_id': 'Action', 'mean': 129.2941176470588},
 {'_id': 'Animation', 'mean': 100.95652173913044},
 {'_id': 'Music', 'mean': 134.25},
 {'_id': 'Thriller', 'mean': 121.03333333333333},
 {'_id': 'Western', 'mean': 149.33333333333334},
 {'_id': 'Drama', 'mean': 132.53142857142856},
 {'_id': 'Sci-Fi', 'mean': 125.8},
 {'_id': 'Comedy', 'mean': 102.47826086956522},
 {'_id': 'Mystery', 'mean': 120.80645161290323},
 {'_id': 'Adventure', 'mean': 127.45},
 {'_id': 'Romance', 'mean': 118.47826086956522}]